In [1]:
import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score


In [2]:
def get_message(mes):
    dic = {}
    dic['message'] = mes.text.split('\n\n')[-2]
    dic['message_id'] = mes.get('id').replace('message', '')
    dic['from_name'] = mes.find(attrs={'class':'from_name'})
    reply = mes.find(attrs = {'class': 'reply_to details'})
    
    # print(reply)
    if reply:
        dic['go_to_message_id'] = reply.find('a').get('href').replace('#go_to_message', '')
    return dic

In [3]:
# папки с ChatExport
sub_list = []
folders = glob.glob("C:/Users/artem/Downloads/Telegram Desktop/ChatExport/ChatExport*")
files = []
for chat_name,  folder in enumerate(folders[:7]):
    files = glob.glob(f"{folder}/*.html")
    messages_all = []
    for file in tqdm(files):
        with open(file, encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            messages = soup.body.find_all(attrs = {'class':'message'})#.text#.split('\n\n')[-2].lower()
        messages_all+=messages
    # chat_name = soup.find(attrs = {'class', "text bold"}).text
    df_sub = pd.DataFrame(list(map(get_message, messages_all)))
    df_sub["chat_name"] = chat_name
    sub_list.append(df_sub)
df = pd.concat(sub_list)

100%|██████████| 321/321 [02:55<00:00,  1.82it/s]


In [4]:
del sub_list

In [5]:
del messages_all

In [6]:
def tokenize(text):
    stop_words = ['за', 'там', 'так', 'то', 'все', 'если', '-',
       ':', 'но', 'как', 'по', 'у', 'есть', 'это', 'я', 'с', 'а', 'не', ')', 'и', '.', 'в', ',']
    tokens = word_tokenize(text.lower(), language="russian")
    tokens = list(filter(lambda x:'посовет' not in x, tokens))
    tokens = list(filter(lambda x: x not in stop_words, tokens))
    
    return list(set(tokens))

In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize
target = 'question'

In [8]:
df['tokens'] = df['message'].apply(tokenize)

In [9]:
target = 'question'
target_list = []
for message in df['message'].values:
    if 'посовет' in message:
        target_list.append(1)
    else:
        target_list.append(0) 
df[target] = target_list

In [10]:
df_reply_mapping = df[~df['go_to_message_id'].isnull()]
df_reply_mapping = df_reply_mapping.rename(columns={"go_to_message_id":"message_id", "message_id": "reply_id"})
df_reply_mapping = pd.merge(df, df_reply_mapping, on=['message_id', 'chat_name'], suffixes=['_question', '_reply'])

In [11]:
df_reply_mapping = df_reply_mapping[df_reply_mapping['question_question']==1]

In [12]:
# df_test.sort_values('log_proba').iloc[-50:]

In [13]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [14]:
# tokens = word_tokenize(row['message'], language="russian")

In [15]:
pd.set_option('display.max_colwidth', None)

In [16]:
pd.options.display.width = 300

In [17]:
df_reply_mapping['message_question_tokens'] = df_reply_mapping['message_question'].apply(tokenize)

In [18]:
df_questions = df[df['question']==1]
df_questions = df_questions[['message','message_id', 'chat_name', 'from_name']]

In [19]:
df_questions['message_tokens'] = df_questions['message'].apply(tokenize)

In [20]:
df_questions.head(2)

,message,message_id,chat_name,from_name,message_tokens
564,"У меня есть текущая работа кем то вроде эникея - сисадмина на частной вилле, работа не пыльная, сменная и достаточно скучная. На объекте есть оборудование - как бы умный дом, автоматизация, видеонаблюдение и все такое. Все бы хорошо если бы не ЗП. В момент мне стало скучно и я вспомнил некогда горячо любимую java, полез в Spring mvc, VUE.js в общем обновлять знания и т.д. с конкретной целью - в поисках лучшей жизни устроиться куда- нибудь джуном: есть куда рости -раз, рост знаний и опыта пропорционален зп- два (поправьте, коль не так). Сижу себе учусь по немногу, тестовые всякие делаю. Тут текущий работодатель это дело видя предлагает поучить SIMPL Control System Programming Software и соответственно ЗП увеличить по итогу. И вот собственно я бы хотел посоветоваться как поступить, понимаю что многие меня знать не знают, но тем не менее, может слышали что то про это чудо (SIMPL). Просто в разработку давно хотелось, но пошел работать сюда и сижу с постоянным чувством что меня нае....ли. Тут мне опять что то предлагают и вроде вкусно, а то что интересно оно вроде уже и почти близко то было... В общем хочу узнать ваши мнения.\n",660,0,None,"[ваши, поступить, предлагает, сисадмина, умный, вот, близко, почти, сменная, зп-, узнать, интересно, хорошо, пыльная, текущая, нибудь, куда, кем, т.д., разработку, сюда, simpl, вкусно, устроиться, хотелось, частной, видя, делаю, эникея, чудо, рост, вилле, тут, работа, control, рости, итогу, programming, горячо, предлагают, слышали, -раз, коль, целью, опыта, java, достаточно, может, опять, знания, жизни, хотел, конкретной, знают, работать, менее, чувством, стало, vue.js, ли, давно, некогда, оно, поисках, поправьте, поучить, меня, лучшей, соответственно, видеонаблюдение, вроде, уже, текущий, system, такое, просто, момент, было, spring, оборудование, два, дом, всякие, обновлять, пошел, себе, скучно, немногу, знать, полез, работодатель, пропорционален, mvc, про, увеличить, знаний, (, нае, общем, скучная, ...]"
1648,"для меня нет авторитетов и кумиров, даже родители, я уже лет в 18 понял что люди которые жили в СССР адекватно не смогут посоветовать или понять как надо жить в настоящем, а тем более знать как что в будущем будет\n",11344,0,None,"[нет, ссср, кумиров, меня, настоящем, 18, или, уже, смогут, люди, более, для, понял, жить, надо, будущем, тем, родители, будет, которые, авторитетов, жили, что, даже, адекватно, лет, понять, знать]"


In [21]:
df_reply_mapping['message_tokens'] = df_reply_mapping['message_question'].apply(tokenize)

In [22]:
df_reply_mapping['key'] = 0
df_questions['key'] = 0

In [23]:
def intersect(row):
    stop_words = {'посоветуйте', 'пожалуйста', 'хорошего', 'хорошую', 'ребята', 'привет',
                  'здравствуйте', 'добрый', 'день', 'кто', 'может', 'посоветуйте', 'кто', 'может',
                 'хорошего', 'мастера',
                  'кто', 'привет', '!', '?', 'посоветовать', 'всем'
                  , 'может', 'хорошую', 'можете', 'добрый',
                'хорошего', 
                  'день', 'добрый', '!', 'посоветуйте',
                 }
    message_question_tokens = set(row['message_question_tokens']) - stop_words
    message_tokens = set(row['message_tokens']) - stop_words
    len1 = len(message_question_tokens&message_tokens)
    len2 = max(len(message_question_tokens), len(message_tokens))
    if len2==0:
        return 0
    return len1/len2

In [24]:
def cosin(row):
    stop_words = {'посоветуйте', 'пожалуйста', 'хорошего', 'хорошую', 'ребята', 'привет',
                  'здравствуйте', 'добрый', 'день', 'кто', 'может', 'посоветуйте', 'кто', 'может',
                 'хорошего', 'мастера',
                  'кто', 'привет', '!', '?', 'посоветовать', 'всем'
                  , 'может', 'хорошую', 'можете', 'добрый',
                'хорошего', 
                  'день', 'добрый', '!', 'посоветуйте',
                 }
    message_question_tokens = set(row['message_question_tokens']) - stop_words
    message_tokens = set(row['message_tokens']) - stop_words
    question_embedding = np.zeros(300)
    message_embedding = np.zeros(300)
    for token in message_question_tokens:
        if token in data:
            question_embedding+=np.array(data[token])
    for token in message_tokens:
        if token in data:
            message_embedding+=np.array(data[token])
    cos_sim = np.dot(question_embedding , message_embedding)/(np.linalg.norm(question_embedding)*np.linalg.norm(message_embedding))
    return cos_sim

In [25]:
# df_reply_mapping

In [26]:
# message_question_tokens

In [29]:
del df

In [30]:
import numpy as np

In [34]:
from functools import reduce

In [35]:
use_tokens = set()
use_tokens |=reduce(lambda x,y: x|y, df_questions['message_tokens'].apply(set).values)
use_tokens |=reduce(lambda x,y: x|y, df_reply_mapping['tokens_question'].apply(set).values)

In [36]:
import io

def load_vectors(fname, use_tokens):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        if tokens[0] in use_tokens:
            data[tokens[0]] = list(map(float, tokens[1:]))
    return data

data = load_vectors("D:/Downloads/cc.ru.300.vec/cc.ru.300.vec", use_tokens)

2000000it [02:03, 16154.73it/s]


In [37]:
df_cross = pd.merge(
    df_reply_mapping[['message_question', 'message_id', 'message_question_tokens', 'key', 'from_name_question', 'from_name_reply']],
    df_questions,
    on='key')

df_cross = df_cross[(df_cross['message_id_x']!=df_cross['message_id_y'])&(df_cross['message_question']!=df_cross['message'])]
df_cross['intersect'] = df_cross.apply(intersect,axis=1)

In [40]:
# df_cross.shape

In [41]:
df_cross['cosin'] = df_cross.apply(cosin,axis=1)

C:\Users\artem\AppData\Local\Temp\ipykernel_15564\1291012260.py:20: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = np.dot(question_embedding , message_embedding)/(np.linalg.norm(question_embedding)*np.linalg.norm(message_embedding))


In [37]:
df_cross = df_cross[df_cross['message_question']!=df_cross['message']]
df_cross = df_cross.sort_values('intersect')

C:\Users\artem\AppData\Local\Temp\ipykernel_15564\1291012260.py:20: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = np.dot(question_embedding , message_embedding)/(np.linalg.norm(question_embedding)*np.linalg.norm(message_embedding))


KeyboardInterrupt: 

In [ ]:
df_cross[df_cross['message_question_tokens'].apply(len)<10]

In [42]:
1

1

In [54]:
df_cross[(df_cross['cosin'].notnull())&(df_cross['cosin']<0.95)].sort_values('cosin').iloc[-180:-120]

,message_question,message_id_x,message_question_tokens,key,from_name_question,from_name_reply,message,message_id_y,chat_name,from_name,message_tokens,intersect,cosin
1252354,"Ребята если есть юрист, который может прокунсултиповать по денежным проблем, посоветуйте пожалуйста или в лс напишите мне\n",96963,"[денежным, или, напишите, прокунсултиповать, который, проблем, юрист, мне, пожалуйста, может, ребята, лс]",0,[\nالكسندر \n ],None,"Народ, всем привет. Есть тут или посоветуйте мастера по кофемашинам. Нужно обслужить, почистить, подремонтировать. Буду благодарен за контакты. Пишите в ЛС\n",271336,4,[\nAF \n ],"[всем, почистить, или, нужно, привет, мастера, пишите, кофемашинам, обслужить, благодарен, тут, народ, подремонтировать, буду, контакты, лс]",0.153846,0.905096
219066,"\nДа ну? Очевидно что человек не умеет пользоваться поиском, иначе бы не задавал таких вопросов. Вы не дали ответа. И не дали ссылки на группу которую посоветовали.\n",9412,"[умеет, вы, человек, очевидно, таких, ?, пользоваться, ну, дали, поиском, ссылки, вопросов, бы, группу, что, иначе, которую, ответа, задавал, на, да]",0,[\nSergey \n ],[\nАня Трай\n ],"А что бы ты мне посоветовал на первое время,ну куда устроиться\n",171458,6,None,"[что, первое, ну, куда, устроиться, бы, на, ты, время, мне]",0.200000,0.905266
217532,"\nДа ну? Очевидно что человек не умеет пользоваться поиском, иначе бы не задавал таких вопросов. Вы не дали ответа. И не дали ссылки на группу которую посоветовали.\n",9412,"[умеет, вы, человек, очевидно, таких, ?, пользоваться, ну, дали, поиском, ссылки, вопросов, бы, группу, что, иначе, которую, ответа, задавал, на, да]",0,[\nSergey \n ],[\nАндрей Артюк\n ],"А что бы ты мне посоветовал на первое время,ну куда устроиться\n",171458,6,None,"[что, первое, ну, куда, устроиться, бы, на, ты, время, мне]",0.200000,0.905266
1131250,Есть на Поляне стоматологическая поликлиника? Или посоветуйте часную стоматологию хорошую\n,34706,"[или, поликлиника, на, часную, хорошую, стоматологию, поляне, ?, стоматологическая]",0,[\nValentin \n ],[\nЕкатерина Хмелевская\n ],"Друзья, кто может посоветовать на красной поляне массаж?\n",297064,4,[\nДенис Тимошевский | финансист\n ],"[массаж, ?, друзья, кто, на, красной, поляне, может]",0.285714,0.906047
1059180,"Друзья, кто может посоветовать на красной поляне массаж?\n",297064,"[массаж, ?, друзья, кто, на, красной, поляне, может]",0,[\nДенис Тимошевский | финансист\n ],None,Есть на Поляне стоматологическая поликлиника? Или посоветуйте часную стоматологию хорошую\n,34706,4,[\nValentin \n ],"[или, поликлиника, на, часную, хорошую, стоматологию, поляне, ?, стоматологическая]",0.285714,0.906047
1056112,"Друзья, кто может посоветовать на красной поляне массаж?\n",297064,"[массаж, ?, друзья, кто, на, красной, поляне, может]",0,[\nДенис Тимошевский | финансист\n ],[\nPatricia Holman\n ],Есть на Поляне стоматологическая поликлиника? Или посоветуйте часную стоматологию хорошую\n,34706,4,[\nValentin \n ],"[или, поликлиника, на, часную, хорошую, стоматологию, поляне, ?, стоматологическая]",0.285714,0.906047
1057646,"Друзья, кто может посоветовать на красной поляне массаж?\n",297064,"[массаж, ?, друзья, кто, на, красной, поляне, может]",0,[\nДенис Тимошевский | финансист\n ],None,Есть на Поляне стоматологическая поликлиника? Или посоветуйте часную стоматологию хорошую\n,34706,4,[\nValentin \n ],"[или, поликлиника, на, часную, хорошую, стоматологию, поляне, ?, стоматологическая]",0.285714,0.906047
1510240,"И сантехника в Поляне хорошего посоветуйте, пожалуйста. Оттепель на потолке 🙏\n",164350,"[потолке, оттепель, на, хорошего, сантехника, пожалуйста, 🙏, поляне]",0,[\nAnna \n ],[\nАлексей Шиверский\n ],А может кто посоветовать инструктора на сноуборде?\n,70864,4,[\n🐾 Катя \n ],"[сноуборде, кто, на, инструктора, ?, может]",0.166667,0.906168
1847950,"Сгорели на пляже посоветуйте, пожалуйста, средство или методЖар\n",226841,"[или, пляже, сгорели, на, средство, пожалуйста, методжар]",0,[\nМаксим Максимович\n ],[\nOlga Fetsova\n ],"

In [ ]:
use_tokens = set()

In [ ]:
len(use_tokens)

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
view = df_cross[(df_cross['intersect'].notnull())&(df_cross['from_name_question']!=df_cross['from_name'])].iloc[-140:-70]

In [ ]:
# use_tokens

In [ ]:
# data

In [ ]:
find_token = 'остеопата'

In [ ]:
# data.keys()

In [ ]:
dic = {}
for token, emb in data.items():
    cos_sim = np.dot(data[token] , data[find_token])/(np.linalg.norm(data[token] )*np.linalg.norm(data[find_token]))
    dic[token] = cos_sim

In [ ]:
pd.Series(dic).sort_values().iloc[-20:]

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import cos

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
pairwise_distances(

In [ ]:
np.array(data[token] - data[find_token]

In [ ]:
for use_tokens 

In [ ]:
len(data)

In [ ]:
data['автошколу']

In [ ]:
aaa

In [ ]:
new_questing = 'посоветуйте автошколу в сочи'

In [ ]:
new_tokens = tokenize(new_questing)

In [ ]:
new_tokens

In [ ]:
df_reply_mapping['message_tokens'] = len(df_reply_mapping)*[new_tokens]

In [ ]:
# df_reply_mapping

In [ ]:
df_reply_mapping['score'] = df_reply_mapping.apply(intersect,axis=1)

In [ ]:
df_reply_mapping.sort_values('score').iloc[-10:][['message_question', 'message_reply','message_question_tokens', 'score']]

In [ ]:
df['tokens'] = df['text'].apply(tokenize)

In [ ]:
df = df[df['tokens'].apply(len)>=3]

In [ ]:
pd.options.display.max_colwidth = 200

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
1

In [ ]:
df_train[df_train['question']==1].sample(n=30)

In [ ]:
train_lenght = len(df_train)

In [ ]:
mean_target = df_train['question'].mean()

In [ ]:
from functools import reduce

In [ ]:
from collections import Counter

In [ ]:
for text in df[df['question']==1]['text'].values:
    pass
    # print(text)
    # print()

In [ ]:
# df['question'].sum()

In [ ]:
from tqdm import tqdm

In [ ]:
total_count = Counter()
for tokens in tqdm(df_train['tokens'].values):
    for token in tokens:
        total_count[token]+=1
    

In [ ]:
# total_count

In [ ]:
target_count = Counter()
for tokens in tqdm( df_train[df_train[target]==1]['tokens'].values):
    for token in tokens:
        target_count[token]+=1

In [ ]:
pd.Series(total_count).sort_values().index[-30:]

In [ ]:
pd.Series(target_count).sort_values()[-30:]

In [ ]:
pd.Series(target_count).sort_values().iloc[-50:].index

In [ ]:
# total_count

In [ ]:
# tokens = df_test['tokens'].iloc[0]

In [ ]:
# pd.Series(target_count.values()).value_counts()

In [ ]:
def predict(tokens):
    score = 0
    for token in tokens:
        p += p*(target_count[token])/(total_count[token]+0.0001)
    return p

In [ ]:
df_train['score'] = df_train['tokens'].apply(predict)

In [ ]:
df_test['score'] = df_test['tokens'].apply(predict)

In [ ]:
# df_test.sort_values('score').iloc[-100:]

In [ ]:
# df_test.sort_values('score')

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(df_train[target], df_train['score'])

In [ ]:
df_test['score'].max()

In [ ]:
vc = df[df[target]==1]['tokens'].apply(len).value_counts()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.bar(vc.index, vc.values)
plt.show()

In [ ]:
# рабоатет даже, если нет слова посоветуйте

In [ ]:
df_test[df_test['tokens'].apply(len)>3].sort_values('score').iloc[-50:]

In [ ]:
# топ признаков того, что это вопрос/просьба подсказать
stat = {}
for key, val in target_count.items():
    if val>=3:
        rate = (val)/(total_count[key]+1)
        stat[key] = rate

In [ ]:
stat['smm']

In [ ]:
# pd.Series(target_count).sort_values().iloc[-100:-50]

In [ ]:
pd.Series(stat).sort_values().iloc[-50:]